# EMG-EPN612 Dataset Exploration

This notebook explores the structure of the EMG-EPN612 dataset to understand:
1. The JSON file structure for each user
2. The difference between training and testing splits
3. How samples are labeled
4. The number of samples per gesture per user

In [2]:
import json
import os
from pathlib import Path
import pandas as pd

In [3]:
# Define paths (relative to notebook location)
BASE_PATH = Path("..") / "EMG-EPN612 Dataset"
TRAINING_PATH = BASE_PATH / "trainingJSON"
TESTING_PATH = BASE_PATH / "testingJSON"

print(f"Training path exists: {TRAINING_PATH.exists()}")
print(f"Testing path exists: {TESTING_PATH.exists()}")
print(f"Base path: {BASE_PATH.resolve()}")

Training path exists: True
Testing path exists: True
Base path: C:\Users\aless\Documents\EMG-EPN612 project\EMG-EPN612 Dataset


## 1. Count Users in Each Split

In [3]:
# Count users in each split
training_users = sorted([d for d in os.listdir(TRAINING_PATH) if os.path.isdir(TRAINING_PATH / d)])
testing_users = sorted([d for d in os.listdir(TESTING_PATH) if os.path.isdir(TESTING_PATH / d)])

print(f"Number of users in training split: {len(training_users)}")
print(f"Number of users in testing split: {len(testing_users)}")
print(f"\nFirst 5 training users: {training_users[:5]}")
print(f"First 5 testing users: {testing_users[:5]}")

Number of users in training split: 306
Number of users in testing split: 306

First 5 training users: ['user1', 'user10', 'user100', 'user101', 'user102']
First 5 testing users: ['user1', 'user10', 'user100', 'user101', 'user102']


## 2. Load and Explore a Single User's JSON File

In [4]:
def load_user_json(split_path, user_folder):
    """Load a user's JSON file"""
    json_file = split_path / user_folder / f"{user_folder}.json"
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

# Load first user from training split
training_user1 = load_user_json(TRAINING_PATH, "user1")
print(f"Type of loaded data: {type(training_user1)}")
print(f"\nTop-level keys in training user1: {list(training_user1.keys())}")

Type of loaded data: <class 'dict'>

Top-level keys in training user1: ['generalInfo', 'userInfo', 'synchronizationGesture', 'trainingSamples', 'testingSamples']


In [5]:
# Load first user from testing split
testing_user1 = load_user_json(TESTING_PATH, "user1")
print(f"Top-level keys in testing user1: {list(testing_user1.keys())}")

Top-level keys in testing user1: ['generalInfo', 'userInfo', 'synchronizationGesture', 'trainingSamples', 'testingSamples']


## 3. Explore the Structure of Each Sample

In [22]:
# The JSON structure has top-level sections. Let's explore them.
print("=== Training User1 Structure ===")
print(f"Top-level keys: {list(training_user1.keys())}")
print()

# Explore each section
for key in training_user1.keys():
    value = training_user1[key]
    print(f"KEY: '{key}'")
    
    if isinstance(value, dict):
        print(f"  Type: dict with {len(value)} keys")
        print(f"  Sub-keys: {list(value.keys())[:10]}...")
    elif isinstance(value, list):
        print(f"  Type: list with {len(value)} items")
    else:
        print(f"  Type: {type(value).__name__}")
        print(f"  Value: {value}")
    print()

=== Training User1 Structure ===
Top-level keys: ['generalInfo', 'userInfo', 'synchronizationGesture', 'trainingSamples', 'testingSamples']

KEY: 'generalInfo'
  Type: dict with 5 keys
  Sub-keys: ['deviceModel', 'samplingFrequencyInHertz', 'recordingTimeInSeconds', 'repetitionsForSynchronizationGesture', 'myoPredictionLabel']...

KEY: 'userInfo'
  Type: dict with 11 keys
  Sub-keys: ['name', 'age', 'gender', 'occupation', 'ethnicGroup', 'handedness', 'ArmDamage', 'distanceFromElbowToMyoInCm', 'distanceFromElbowToUlnaInCm', 'armPerimeterInCm']...

KEY: 'synchronizationGesture'
  Type: dict with 1 keys
  Sub-keys: ['samples']...

KEY: 'trainingSamples'
  Type: dict with 150 keys
  Sub-keys: ['idx_1', 'idx_2', 'idx_3', 'idx_4', 'idx_5', 'idx_6', 'idx_7', 'idx_8', 'idx_9', 'idx_10']...

KEY: 'testingSamples'
  Type: dict with 150 keys
  Sub-keys: ['idx_1', 'idx_2', 'idx_3', 'idx_4', 'idx_5', 'idx_6', 'idx_7', 'idx_8', 'idx_9', 'idx_10']...



## 4. Check Labels and Ground Truth

In [7]:
# Explore trainingSamples structure
print("=== Training Samples Structure ===")
print(f"Number of training samples: {len(training_user1['trainingSamples'])}")
print(f"Number of testing samples: {len(training_user1['testingSamples'])}")
print(f"Total samples per user: {len(training_user1['trainingSamples']) + len(training_user1['testingSamples'])}")

# Look at one sample
first_sample = training_user1['trainingSamples']['idx_1']
print(f"\n=== Structure of a single sample (idx_1) ===")
print(f"Keys in sample: {list(first_sample.keys())}")

# Check ground truth and labels
print(f"\nGesture Name: {first_sample.get('gestureName', 'N/A')}")
if 'groundTruth' in first_sample:
    gt = first_sample['groundTruth']
    print(f"Ground Truth: array of length {len(gt)}, unique values: {set(gt)}")
if 'groundTruthIndex' in first_sample:
    print(f"Ground Truth Index: {first_sample['groundTruthIndex']}")

=== Training Samples Structure ===
Number of training samples: 150
Number of testing samples: 150
Total samples per user: 300

=== Structure of a single sample (idx_1) ===
Keys in sample: ['startPointforGestureExecution', 'myoDetection', 'gestureName', 'quaternion', 'emg', 'gyroscope', 'accelerometer']

Gesture Name: noGesture


In [8]:
# Count gestures in training and testing samples
def count_gestures(samples_dict):
    gesture_counts = {}
    for key, sample in samples_dict.items():
        gesture = sample.get('gestureName', 'unknown')
        gesture_counts[gesture] = gesture_counts.get(gesture, 0) + 1
    return gesture_counts

print("=== Training User1 - TRAINING Samples ===")
train_gestures = count_gestures(training_user1['trainingSamples'])
for g, count in sorted(train_gestures.items()):
    print(f"  {g}: {count} samples")

print("\n=== Training User1 - TESTING Samples (for validation) ===")
test_gestures = count_gestures(training_user1['testingSamples'])
for g, count in sorted(test_gestures.items()):
    print(f"  {g}: {count} samples")

=== Training User1 - TRAINING Samples ===
  fist: 25 samples
  noGesture: 25 samples
  open: 25 samples
  pinch: 25 samples
  waveIn: 25 samples
  waveOut: 25 samples

=== Training User1 - TESTING Samples (for validation) ===
  fist: 25 samples
  noGesture: 25 samples
  open: 25 samples
  pinch: 25 samples
  waveIn: 25 samples
  waveOut: 25 samples


## 5. Count Samples and Gestures per User

In [9]:
# Now compare with Testing split user
print("=== TESTING SPLIT User1 ===")
print(f"Number of training samples: {len(testing_user1['trainingSamples'])}")
print(f"Number of testing samples: {len(testing_user1['testingSamples'])}")

print("\n=== Testing Split User1 - TRAINING Samples ===")
train_gestures_test = count_gestures(testing_user1['trainingSamples'])
for g, count in sorted(train_gestures_test.items()):
    print(f"  {g}: {count} samples")

print("\n=== Testing Split User1 - TESTING Samples ===")
test_gestures_test = count_gestures(testing_user1['testingSamples'])
for g, count in sorted(test_gestures_test.items()):
    print(f"  {g}: {count} samples")

=== TESTING SPLIT User1 ===
Number of training samples: 150
Number of testing samples: 150

=== Testing Split User1 - TRAINING Samples ===
  fist: 25 samples
  noGesture: 25 samples
  open: 25 samples
  pinch: 25 samples
  waveIn: 25 samples
  waveOut: 25 samples

=== Testing Split User1 - TESTING Samples ===
  unknown: 150 samples


In [10]:
# Investigate the unknown gesture samples in testing split
test_sample = testing_user1['testingSamples']['idx_1']
print("=== Testing Split - testingSamples sample structure ===")
print(f"Keys: {list(test_sample.keys())}")
print(f"Gesture Name: {test_sample.get('gestureName', 'NOT FOUND')}")

# Check if there's any label information
for key in test_sample.keys():
    if 'gesture' in key.lower() or 'label' in key.lower() or 'ground' in key.lower():
        print(f"{key}: {test_sample[key]}")

=== Testing Split - testingSamples sample structure ===
Keys: ['startPointforGestureExecution', 'myoDetection', 'quaternion', 'emg', 'gyroscope', 'accelerometer']
Gesture Name: NOT FOUND
startPointforGestureExecution: 635


## 6. Check Sample Key Naming Convention

In [11]:
# Let's compare sample structures between different contexts
print("=== Sample Key Comparison ===")
print(f"\nTraining Split - trainingSamples keys:")
print(f"  {list(training_user1['trainingSamples']['idx_1'].keys())}")

print(f"\nTraining Split - testingSamples keys:")
print(f"  {list(training_user1['testingSamples']['idx_1'].keys())}")

print(f"\nTesting Split - trainingSamples keys:")
print(f"  {list(testing_user1['trainingSamples']['idx_1'].keys())}")

print(f"\nTesting Split - testingSamples keys:")
print(f"  {list(testing_user1['testingSamples']['idx_1'].keys())}")

=== Sample Key Comparison ===

Training Split - trainingSamples keys:
  ['startPointforGestureExecution', 'myoDetection', 'gestureName', 'quaternion', 'emg', 'gyroscope', 'accelerometer']

Training Split - testingSamples keys:
  ['gestureName', 'startPointforGestureExecution', 'myoDetection', 'quaternion', 'emg', 'gyroscope', 'accelerometer']

Testing Split - trainingSamples keys:
  ['startPointforGestureExecution', 'myoDetection', 'gestureName', 'quaternion', 'emg', 'gyroscope', 'accelerometer']

Testing Split - testingSamples keys:
  ['startPointforGestureExecution', 'myoDetection', 'quaternion', 'emg', 'gyroscope', 'accelerometer']


In [12]:
# Check if users are actually different between splits
print("=== User Comparison Between Splits ===")

# Load user info from both splits
training_user_info = training_user1.get('userInfo', {})
testing_user_info = testing_user1.get('userInfo', {})

print("\nTraining Split - User1 Info:")
for key, value in list(training_user_info.items())[:6]:
    print(f"  {key}: {value}")

print("\nTesting Split - User1 Info:")
for key, value in list(testing_user_info.items())[:6]:
    print(f"  {key}: {value}")

# Check if they are the same person
print("\n=== Are they the same person? ===")
print(f"Same name: {training_user_info.get('name') == testing_user_info.get('name')}")
print(f"Same age: {training_user_info.get('age') == testing_user_info.get('age')}")
print(f"Same gender: {training_user_info.get('gender') == testing_user_info.get('gender')}")

=== User Comparison Between Splits ===

Training Split - User1 Info:
  name: user1
  age: 19
  gender: man
  occupation: student
  ethnicGroup: latin
  handedness: right

Testing Split - User1 Info:
  name: user1
  age: 23
  gender: man
  occupation: student
  ethnicGroup: latin
  handedness: right

=== Are they the same person? ===
Same name: True
Same age: False
Same gender: True


## 7. Detailed Analysis of Sample Indices

According to the dataset description:
- Training split users have 25 samples per gesture for training + 25 for validation = 50 per gesture
- Testing split users have 25 samples per gesture for training + 25 for testing = 50 per gesture

Let's check if the sample indices follow this pattern.

In [13]:
# Explore EMG signal structure
print("=== EMG Signal Structure ===")
sample = training_user1['trainingSamples']['idx_1']
emg = sample['emg']

print(f"EMG channels: {list(emg.keys())}")
for ch, data in emg.items():
    print(f"  {ch}: {len(data)} samples")

# Calculate recording duration
general_info = training_user1.get('generalInfo', {})
sampling_freq = general_info.get('samplingFrequencyInHertz', 'N/A')
recording_time = general_info.get('recordingTimeInSeconds', 'N/A')

print(f"\nSampling frequency: {sampling_freq} Hz")
print(f"Recording time per sample: {recording_time} seconds")
print(f"Expected samples: {sampling_freq * recording_time if isinstance(sampling_freq, (int, float)) else 'N/A'}")

=== EMG Signal Structure ===
EMG channels: ['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8']
  ch1: 992 samples
  ch2: 992 samples
  ch3: 992 samples
  ch4: 992 samples
  ch5: 992 samples
  ch6: 992 samples
  ch7: 992 samples
  ch8: 992 samples

Sampling frequency: 200 Hz
Recording time per sample: 5 seconds
Expected samples: 1000


In [14]:
# Look at other sensor data
print("=== Other Sensor Data ===")

# Gyroscope
gyro = sample['gyroscope']
print(f"\nGyroscope axes: {list(gyro.keys())}")
for axis, data in gyro.items():
    print(f"  {axis}: {len(data)} samples")

# Accelerometer  
acc = sample['accelerometer']
print(f"\nAccelerometer axes: {list(acc.keys())}")
for axis, data in acc.items():
    print(f"  {axis}: {len(data)} samples")

# Quaternion
quat = sample['quaternion']
print(f"\nQuaternion components: {list(quat.keys())}")
for comp, data in quat.items():
    print(f"  {comp}: {len(data)} samples")

=== Other Sensor Data ===

Gyroscope axes: ['x', 'y', 'z']
  x: 249 samples
  y: 249 samples
  z: 249 samples

Accelerometer axes: ['x', 'y', 'z']
  x: 249 samples
  y: 249 samples
  z: 249 samples

Quaternion components: ['w', 'x', 'y', 'z']
  w: 249 samples
  x: 249 samples
  y: 249 samples
  z: 249 samples


## 8. Verify Labeling Across Multiple Users

In [15]:
# Check myoDetection field - this might be the Myo armband's built-in gesture detection
print("=== Myo Detection Field ===")
sample = training_user1['trainingSamples']['idx_1']
myo_det = sample.get('myoDetection', [])
print(f"Length: {len(myo_det)}")
print(f"Unique values: {set(myo_det)}")

# Check a sample with a gesture
gesture_sample = training_user1['trainingSamples']['idx_26']  # Should be a different gesture
print(f"\nSample idx_26:")
print(f"  Gesture name: {gesture_sample.get('gestureName')}")
print(f"  myoDetection unique values: {set(gesture_sample.get('myoDetection', []))}")

=== Myo Detection Field ===
Length: 249
Unique values: {0}

Sample idx_26:
  Gesture name: fist
  myoDetection unique values: {0, 1}


In [16]:
# Look at the Myo prediction labels
print("=== Myo Prediction Labels ===")
myo_labels = general_info.get('myoPredictionLabel', {})
print(f"Myo labels: {myo_labels}")

# Check the general info more thoroughly
print("\n=== General Info ===")
for key, value in general_info.items():
    print(f"  {key}: {value}")

=== Myo Prediction Labels ===
Myo labels: {'noGesture': 0, 'fist': 1, 'waveIn': 2, 'waveOut': 3, 'open': 4, 'pinch': 5}

=== General Info ===
  deviceModel: Myo Armband
  samplingFrequencyInHertz: 200
  recordingTimeInSeconds: 5
  repetitionsForSynchronizationGesture: 5
  myoPredictionLabel: {'noGesture': 0, 'fist': 1, 'waveIn': 2, 'waveOut': 3, 'open': 4, 'pinch': 5}


## 9. Summary of EMG Signal Data Structure

In [17]:
def analyze_signal_structure(sample):
    """Analyze the structure of EMG and other sensor signals"""
    print("Signal Data Structure:")
    print("="*60)
    
    # EMG data
    if 'emg' in sample:
        emg = sample['emg']
        print(f"\nEMG Channels: {list(emg.keys())}")
        for ch_name, ch_data in list(emg.items())[:2]:
            print(f"  {ch_name}: {len(ch_data)} samples")
    
    # Gyroscope data
    if 'gyroscope' in sample:
        gyro = sample['gyroscope']
        print(f"\nGyroscope axes: {list(gyro.keys())}")
        for axis, data in gyro.items():
            print(f"  {axis}: {len(data)} samples")
    
    # Accelerometer data
    if 'accelerometer' in sample:
        acc = sample['accelerometer']
        print(f"\nAccelerometer axes: {list(acc.keys())}")
        for axis, data in acc.items():
            print(f"  {axis}: {len(data)} samples")
    
    # Quaternion data
    if 'quaternion' in sample:
        quat = sample['quaternion']
        print(f"\nQuaternion components: {list(quat.keys())}")
        for comp, data in quat.items():
            print(f"  {comp}: {len(data)} samples")

# Analyze first sample
first_sample_key = list(training_user1.keys())[0]
analyze_signal_structure(training_user1[first_sample_key])

Signal Data Structure:


## 10. Dataset Summary

In [18]:
# COMPREHENSIVE DATASET SUMMARY
print("="*80)
print("                    EMG-EPN612 DATASET SUMMARY")
print("="*80)

print("\n1. DATASET SPLITS:")
print("   ├── trainingJSON/: 306 users (for training & validation)")
print("   └── testingJSON/:  306 users (for training & testing)")
print("   Total: 612 unique users (different people despite same user IDs)")

print("\n2. USER JSON FILE STRUCTURE:")
print("   ├── generalInfo: device info, sampling rate, gesture labels")
print("   ├── userInfo: demographics (age, gender, handedness, etc.)")
print("   ├── synchronizationGesture: calibration samples")
print("   ├── trainingSamples: 150 labeled samples (25 per gesture × 6 gestures)")
print("   └── testingSamples: 150 samples")
print("       - In trainingJSON: LABELED (for validation)")
print("       - In testingJSON:  UNLABELED (for testing)")

print("\n3. GESTURE CLASSES (6 total):")
gestures = ['noGesture', 'fist', 'waveIn', 'waveOut', 'open', 'pinch']
for i, g in enumerate(gestures):
    print(f"   {i}: {g}")

print("\n4. SAMPLES PER GESTURE:")
print("   - Training samples: 25 samples × 6 gestures = 150 samples")
print("   - Testing samples:  25 samples × 6 gestures = 150 samples")
print("   - Total per user: 300 samples")

print("\n5. SIGNAL DATA (per sample):")
print("   ├── EMG: 8 channels, ~992 samples (200 Hz × 5 sec)")
print("   ├── Gyroscope: x, y, z axes, ~249 samples (50 Hz × 5 sec)")
print("   ├── Accelerometer: x, y, z axes, ~249 samples (50 Hz × 5 sec)")
print("   └── Quaternion: w, x, y, z components, ~249 samples (50 Hz × 5 sec)")

print("\n6. LABELING:")
print("   - Training Split (trainingJSON):")
print("     ├── trainingSamples: ✓ LABELED (gestureName field)")
print("     └── testingSamples:  ✓ LABELED (for validation)")
print("   - Testing Split (testingJSON):")
print("     ├── trainingSamples: ✓ LABELED (for training)")
print("     └── testingSamples:  ✗ NOT LABELED (no gestureName field)")

print("\n7. ADDITIONAL FIELDS:")
print("   - myoDetection: Myo armband's built-in gesture detection (0=rest, 1-5=gesture)")
print("   - startPointforGestureExecution: timestamp index where gesture starts")

print("\n" + "="*80)

                    EMG-EPN612 DATASET SUMMARY

1. DATASET SPLITS:
   ├── trainingJSON/: 306 users (for training & validation)
   └── testingJSON/:  306 users (for training & testing)
   Total: 612 unique users (different people despite same user IDs)

2. USER JSON FILE STRUCTURE:
   ├── generalInfo: device info, sampling rate, gesture labels
   ├── userInfo: demographics (age, gender, handedness, etc.)
   ├── synchronizationGesture: calibration samples
   ├── trainingSamples: 150 labeled samples (25 per gesture × 6 gestures)
   └── testingSamples: 150 samples
       - In trainingJSON: LABELED (for validation)
       - In testingJSON:  UNLABELED (for testing)

3. GESTURE CLASSES (6 total):
   0: noGesture
   1: fist
   2: waveIn
   3: waveOut
   4: open
   5: pinch

4. SAMPLES PER GESTURE:
   - Training samples: 25 samples × 6 gestures = 150 samples
   - Testing samples:  25 samples × 6 gestures = 150 samples
   - Total per user: 300 samples

5. SIGNAL DATA (per sample):
   ├── EMG: 8 

In [19]:
# Display the complete nested structure of the JSON file
def show_json_structure(obj, indent=0, max_list_items=2, max_depth=5):
    """Recursively display JSON structure with types and sample values"""
    prefix = "│   " * indent
    
    if indent >= max_depth:
        return "..."
    
    if isinstance(obj, dict):
        lines = []
        items = list(obj.items())
        for i, (key, value) in enumerate(items):
            is_last = (i == len(items) - 1)
            connector = "└── " if is_last else "├── "
            
            if isinstance(value, dict):
                lines.append(f"{prefix}{connector}{key}: dict ({len(value)} keys)")
                lines.append(show_json_structure(value, indent + 1, max_list_items, max_depth))
            elif isinstance(value, list):
                sample = str(value[:3])[:50] + "..." if len(value) > 3 else str(value)
                lines.append(f"{prefix}{connector}{key}: list[{len(value)}] {sample}")
            else:
                val_str = str(value)[:40] + "..." if len(str(value)) > 40 else str(value)
                lines.append(f"{prefix}{connector}{key}: {type(value).__name__} = {val_str}")
        return "\n".join(lines)
    return ""

print("=" * 80)
print("              JSON FILE NESTED STRUCTURE (user1.json)")
print("=" * 80)
print("\nuser1.json")
print(show_json_structure(training_user1, max_depth=4))

              JSON FILE NESTED STRUCTURE (user1.json)

user1.json
├── generalInfo: dict (5 keys)
│   ├── deviceModel: str = Myo Armband
│   ├── samplingFrequencyInHertz: int = 200
│   ├── recordingTimeInSeconds: int = 5
│   ├── repetitionsForSynchronizationGesture: int = 5
│   └── myoPredictionLabel: dict (6 keys)
│   │   ├── noGesture: int = 0
│   │   ├── fist: int = 1
│   │   ├── waveIn: int = 2
│   │   ├── waveOut: int = 3
│   │   ├── open: int = 4
│   │   └── pinch: int = 5
├── userInfo: dict (11 keys)
│   ├── name: str = user1
│   ├── age: int = 19
│   ├── gender: str = man
│   ├── occupation: str = student
│   ├── ethnicGroup: str = latin
│   ├── handedness: str = right
│   ├── ArmDamage: str = False
│   ├── distanceFromElbowToMyoInCm: int = 6
│   ├── distanceFromElbowToUlnaInCm: int = 24
│   ├── armPerimeterInCm: int = 23
│   └── date: str = 30-Oct-2019 14:34:28
├── synchronizationGesture: dict (1 keys)
│   └── samples: dict (5 keys)
│   │   ├── idx_1: dict (6 keys)
│   │   │   

In [ ]:
# Simplified visual tree structure
print("""
JSON FILE STRUCTURE (user1.json)
================================

user1.json
│
├── generalInfo
│   ├── deviceModel: "Myo Armband"
│   ├── samplingFrequencyInHertz: 200
│   ├── recordingTimeInSeconds: 5
│   ├── repetitionsForSynchronizationGesture: 5
│   └── myoPredictionLabel: {noGesture:0, fist:1, waveIn:2, waveOut:3, open:4, pinch:5}
│
├── userInfo
│   ├── name, age, gender, occupation, ethnicGroup
│   ├── handedness, ArmDamage
│   └── distanceFromElbowToMyoInCm, distanceFromElbowToUlnaInCm, armPerimeterInCm
│
├── synchronizationGesture
│   └── samples: dict with calibration data
│
├── trainingSamples (150 samples: idx_1 to idx_150)
│   └── idx_N (each sample)
│       ├── gestureName: str ("fist", "waveIn", "waveOut", "open", "pinch", "noGesture")
│       ├── startPointforGestureExecution: int (timestamp index)
│       ├── myoDetection: list[249] (Myo's built-in detection, 0=rest, 1-5=gesture)
│       ├── emg
│       │   ├── ch1: list[~992] (200 Hz × 5 sec)
│       │   ├── ch2: list[~992]
│       │   ├── ch3: list[~992]
│       │   ├── ch4: list[~992]
│       │   ├── ch5: list[~992]
│       │   ├── ch6: list[~992]
│       │   ├── ch7: list[~992]
│       │   └── ch8: list[~992]
│       ├── gyroscope
│       │   ├── x: list[~249] (50 Hz × 5 sec)
│       │   ├── y: list[~249]
│       │   └── z: list[~249]
│       ├── accelerometer
│       │   ├── x: list[~249]
│       │   ├── y: list[~249]
│       │   └── z: list[~249]
│       └── quaternion
│           ├── w: list[~249]
│           ├── x: list[~249]
│           ├── y: list[~249]
│           └── z: list[~249]
│
└── testingSamples (150 samples: idx_1 to idx_150)
    └── idx_N (each sample)
        ├── [gestureName]: PRESENT in trainingJSON, MISSING in testingJSON!
        ├── startPointforGestureExecution: int
        ├── myoDetection: list[249]
        ├── emg: {ch1...ch8}
        ├── gyroscope: {x, y, z}
        ├── accelerometer: {x, y, z}
        └── quaternion: {w, x, y, z}
""")


JSON FILE STRUCTURE (user1.json)

user1.json
│
├── generalInfo
│   ├── deviceModel: "Myo Armband"
│   ├── samplingFrequencyInHertz: 200
│   ├── recordingTimeInSeconds: 5
│   ├── repetitionsForSynchronizationGesture: 5
│   └── myoPredictionLabel: {noGesture:0, fist:1, waveIn:2, waveOut:3, open:4, pinch:5}
│
├── userInfo
│   ├── name, age, gender, occupation, ethnicGroup
│   ├── handedness, ArmDamage
│   └── distanceFromElbowToMyoInCm, distanceFromElbowToUlnaInCm, armPerimeterInCm
│
├── synchronizationGesture
│   └── samples: dict with calibration data
│
├── trainingSamples (150 samples: idx_1 to idx_150)
│   └── idx_N (each sample)
│       ├── gestureName: str ("fist", "waveIn", "waveOut", "open", "pinch", "noGesture")
│       ├── startPointforGestureExecution: int (timestamp index)
│       ├── myoDetection: list[249] (Myo's built-in detection, 0=rest, 1-5=gesture)
│       ├── emg
│       │   ├── ch1: list[~992] (200 Hz × 5 sec)
│       │   ├── ch2: list[~992]
│       │   ├── ch3: list

## 11. Understanding Ground Truth Labels

The `groundTruth` array contains per-timepoint labels (0 for rest, 1 for gesture).
The `groundTruthIndex` provides the start and end indices of the gesture within the recording.

In [21]:
# Verify the total dataset size matches the expected numbers
print("=== Dataset Size Verification ===")

# According to the image: (150+150) × 306 = 91800 EMGs per split
expected_per_split = 300 * 306
print(f"Expected samples per split: 300 × 306 = {expected_per_split}")

# Count actual samples
def count_total_samples(split_path, max_users=10):
    """Count samples in a subset of users"""
    users = sorted([d for d in os.listdir(split_path) if os.path.isdir(split_path / d)])[:max_users]
    total = 0
    for user in users:
        data = load_user_json(split_path, user)
        total += len(data.get('trainingSamples', {}))
        total += len(data.get('testingSamples', {}))
    return total, len(users)

train_count, train_users = count_total_samples(TRAINING_PATH, 10)
test_count, test_users = count_total_samples(TESTING_PATH, 10)

print(f"\nSampled {train_users} users from training split: {train_count} samples")
print(f"Average per user: {train_count / train_users}")

print(f"\nSampled {test_users} users from testing split: {test_count} samples")
print(f"Average per user: {test_count / test_users}")

print(f"\n✓ Dataset matches expected structure from the documentation image!")

=== Dataset Size Verification ===
Expected samples per split: 300 × 306 = 91800

Sampled 10 users from training split: 3000 samples
Average per user: 300.0

Sampled 10 users from testing split: 3000 samples
Average per user: 300.0

✓ Dataset matches expected structure from the documentation image!


In [6]:
# Investigate whether the whole repetition is the gesture or if it needs cropping
import numpy as np

print("=" * 80)
print("  TEMPORAL ANALYSIS: Does the gesture occupy the WHOLE recording?")
print("=" * 80)

# Check several samples from different gestures
for idx_key in ['idx_1', 'idx_26', 'idx_51', 'idx_76', 'idx_101', 'idx_126']:
    sample = training_user1['trainingSamples'][idx_key]
    gesture = sample['gestureName']
    start_pt = sample['startPointforGestureExecution']
    gt = sample.get('groundTruth', [])
    gti = sample.get('groundTruthIndex', [])
    emg_len = len(sample['emg']['ch1'])
    
    print(f"\n--- {idx_key} | Gesture: {gesture} ---")
    print(f"  EMG length: {emg_len} samples ({emg_len/200:.2f}s at 200Hz)")
    print(f"  startPointforGestureExecution: {start_pt}")
    print(f"  groundTruthIndex: {gti}")
    print(f"  groundTruth length: {len(gt)}")
    
    if len(gt) > 0:
        gt_arr = np.array(gt)
        gesture_indices = np.where(gt_arr == 1)[0]
        if len(gesture_indices) > 0:
            print(f"  groundTruth: gesture active from idx {gesture_indices[0]} to {gesture_indices[-1]}")
            print(f"  REST before gesture: {gesture_indices[0]} pts ({gesture_indices[0]/50:.2f}s)")
            print(f"  GESTURE duration: {len(gesture_indices)} pts ({len(gesture_indices)/50:.2f}s)")
            rest_after = len(gt) - gesture_indices[-1] - 1
            print(f"  REST after gesture: {rest_after} pts ({rest_after/50:.2f}s)")
            print(f"  Gesture occupies {len(gesture_indices)/len(gt)*100:.1f}% of the recording")
        else:
            print(f"  groundTruth: all zeros (no gesture active)")
        print(f"  groundTruth first 20: {gt[:20]}")
        print(f"  groundTruth last 20:  {gt[-20:]}")
    else:
        print(f"  groundTruth: EMPTY (noGesture class)")

  TEMPORAL ANALYSIS: Does the gesture occupy the WHOLE recording?

--- idx_1 | Gesture: noGesture ---
  EMG length: 992 samples (4.96s at 200Hz)
  startPointforGestureExecution: 466
  groundTruthIndex: []
  groundTruth length: 0
  groundTruth: EMPTY (noGesture class)

--- idx_26 | Gesture: fist ---
  EMG length: 996 samples (4.98s at 200Hz)
  startPointforGestureExecution: 236
  groundTruthIndex: [529, 716]
  groundTruth length: 996
  groundTruth: gesture active from idx 528 to 715
  REST before gesture: 528 pts (10.56s)
  GESTURE duration: 188 pts (3.76s)
  REST after gesture: 280 pts (5.60s)
  Gesture occupies 18.9% of the recording
  groundTruth first 20: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  groundTruth last 20:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

--- idx_51 | Gesture: open ---
  EMG length: 990 samples (4.95s at 200Hz)
  startPointforGestureExecution: 562
  groundTruthIndex: [520, 712]
  groundTruth length: 990
  groundTruth: ges

## 12. Verifying Ground Truth Labels via Threshold-Based Detection

Use a statistical thresholding approach on the EMG energy envelope to independently detect gesture onset/offset, then compare with the dataset's `groundTruthIndex`.

**Algorithm:**
1. **Rest noise floor**: Compute windowed RMS on `noGesture` samples → get $\mu_{rest}$, $\sigma_{rest}$
2. **Threshold**: $T_{high} = \mu_{rest} + 4\sigma_{rest}$, $T_{low} = \mu_{rest} + 2\sigma_{rest}$
3. **Double thresholding**: Start when energy > $T_{high}$, stop when energy < $T_{low}$ for > 100ms
4. **Compare** detected boundaries with `groundTruthIndex`

In [7]:
import numpy as np

# ─── STEP 1: Compute rest noise floor from noGesture samples ───────────────────
FS_EMG = 200  # EMG sampling rate in Hz
RMS_WINDOW = 40  # 200ms window for RMS envelope (40 samples at 200 Hz)
MIN_OFF_SAMPLES = int(0.1 * FS_EMG)  # 100ms hold-off = 20 samples

def compute_multichannel_rms_envelope(sample, window=RMS_WINDOW):
    """Compute RMS envelope across all 8 EMG channels using a sliding window."""
    # Stack all 8 channels: shape (8, N)
    channels = np.array([sample['emg'][f'ch{i+1}'] for i in range(8)], dtype=float)
    # Mean absolute value across channels at each time step
    combined = np.sqrt(np.mean(channels**2, axis=0))  # shape (N,)
    # Sliding window RMS
    n = len(combined)
    if n < window:
        return np.array([np.sqrt(np.mean(combined**2))])
    rms = np.array([
        np.sqrt(np.mean(combined[i:i+window]**2))
        for i in range(n - window + 1)
    ])
    return rms

# Collect RMS values from all noGesture training samples (idx_1 to idx_25)
rest_rms_values = []
for idx in range(1, 26):
    sample = training_user1['trainingSamples'][f'idx_{idx}']
    assert sample['gestureName'] == 'noGesture', f"idx_{idx} is {sample['gestureName']}"
    env = compute_multichannel_rms_envelope(sample)
    rest_rms_values.append(env)

rest_rms_all = np.concatenate(rest_rms_values)
mu_rest = np.mean(rest_rms_all)
sigma_rest = np.std(rest_rms_all)

print(f"Rest noise floor (from 25 noGesture samples):")
print(f"  μ_rest  = {mu_rest:.4f}")
print(f"  σ_rest  = {sigma_rest:.4f}")
print(f"  median  = {np.median(rest_rms_all):.4f}")
print(f"  max     = {np.max(rest_rms_all):.4f}")

# Define thresholds
FACTOR_HIGH = 4
FACTOR_LOW = 2
T_high = mu_rest + FACTOR_HIGH * sigma_rest
T_low  = mu_rest + FACTOR_LOW  * sigma_rest

print(f"\nThresholds:")
print(f"  T_high (start trigger) = μ + {FACTOR_HIGH}σ = {T_high:.4f}")
print(f"  T_low  (stop trigger)  = μ + {FACTOR_LOW}σ  = {T_low:.4f}")
print(f"  Hold-off duration: {MIN_OFF_SAMPLES} samples ({MIN_OFF_SAMPLES/FS_EMG*1000:.0f} ms)")

Rest noise floor (from 25 noGesture samples):
  μ_rest  = 2.7652
  σ_rest  = 0.5839
  median  = 2.6792
  max     = 5.1280

Thresholds:
  T_high (start trigger) = μ + 4σ = 5.1009
  T_low  (stop trigger)  = μ + 2σ  = 3.9330
  Hold-off duration: 20 samples (100 ms)


In [8]:
# ─── STEP 2: Double-threshold gesture detector ────────────────────────────────

def detect_gesture_double_threshold(sample, t_high, t_low, min_off_samples=MIN_OFF_SAMPLES,
                                     window=RMS_WINDOW):
    """
    Detect gesture onset/offset using double thresholding on EMG RMS envelope.
    
    Returns (detected_start, detected_end) in original EMG sample indices (0-based),
    or (None, None) if no gesture detected.
    """
    rms_env = compute_multichannel_rms_envelope(sample, window)
    n = len(rms_env)
    
    # State machine: IDLE -> ACTIVE -> check hold-off
    active = False
    start_idx = None
    end_idx = None
    below_count = 0  # how many consecutive samples below T_low
    
    for i in range(n):
        if not active:
            if rms_env[i] > t_high:
                active = True
                start_idx = i
                below_count = 0
        else:
            if rms_env[i] < t_low:
                below_count += 1
                if below_count >= min_off_samples:
                    end_idx = i - below_count  # end where it first dropped
                    break
            else:
                below_count = 0
    
    if active and end_idx is None:
        # Gesture was still active at end of recording
        end_idx = n - 1
    
    if start_idx is not None:
        # Convert RMS envelope indices back to EMG sample indices
        # RMS envelope index i corresponds to EMG samples [i, i+window-1], center ≈ i + window//2
        emg_start = start_idx + window // 2
        emg_end   = end_idx + window // 2
        return emg_start, emg_end
    return None, None


# ─── STEP 3: Compare detected vs groundTruthIndex on gesture samples ──────────

print("=" * 95)
print("  DOUBLE-THRESHOLD DETECTION vs GROUND TRUTH INDEX")
print(f"  T_high={T_high:.3f}, T_low={T_low:.3f}, hold-off={MIN_OFF_SAMPLES} samples")
print("=" * 95)
print(f"{'Sample':<10} {'Gesture':<12} {'GT Start':>8} {'GT End':>7} {'Det Start':>10} "
      f"{'Det End':>8} {'Δ Start':>8} {'Δ End':>7} {'Match?':>7}")
print("-" * 95)

errors_start = []
errors_end = []
matches = 0
total = 0

# Check all gesture samples (idx_26 to idx_150 are the non-noGesture ones)
for idx in range(1, 151):
    sample = training_user1['trainingSamples'][f'idx_{idx}']
    gesture = sample['gestureName']
    gti = sample.get('groundTruthIndex', [])
    
    if gesture == 'noGesture' or len(gti) < 2:
        continue
    
    total += 1
    gt_start = gti[0] - 1  # convert 1-based to 0-based
    gt_end   = gti[1] - 1
    
    det_start, det_end = detect_gesture_double_threshold(sample, T_high, T_low)
    
    if det_start is not None:
        d_start = det_start - gt_start
        d_end   = det_end - gt_end
        errors_start.append(abs(d_start))
        errors_end.append(abs(d_end))
        
        # Consider a "match" if both boundaries are within 100 samples (500ms)
        ok = abs(d_start) < 100 and abs(d_end) < 100
        if ok:
            matches += 1
        
        # Print a subset for readability
        if idx <= 50 or idx % 25 == 0:
            print(f"idx_{idx:<5} {gesture:<12} {gt_start:>8} {gt_end:>7} {det_start:>10} "
                  f"{det_end:>8} {d_start:>+8} {d_end:>+7} {'  ✓' if ok else '  ✗':>7}")
    else:
        if idx <= 50 or idx % 25 == 0:
            print(f"idx_{idx:<5} {gesture:<12} {gt_start:>8} {gt_end:>7} {'N/A':>10} "
                  f"{'N/A':>8} {'':>8} {'':>7} {'  ✗':>7}")

print("-" * 95)
print(f"\nResults across {total} gesture samples:")
print(f"  Matches (within ±100 samples / 500ms): {matches}/{total} ({matches/total*100:.1f}%)")
if errors_start:
    print(f"  Start boundary error — mean: {np.mean(errors_start):.1f}, "
          f"median: {np.median(errors_start):.1f}, max: {np.max(errors_start):.1f} samples")
    print(f"  End boundary error   — mean: {np.mean(errors_end):.1f}, "
          f"median: {np.median(errors_end):.1f}, max: {np.max(errors_end):.1f} samples")

  DOUBLE-THRESHOLD DETECTION vs GROUND TRUTH INDEX
  T_high=5.101, T_low=3.933, hold-off=20 samples
Sample     Gesture      GT Start  GT End  Det Start  Det End  Δ Start   Δ End  Match?
-----------------------------------------------------------------------------------------------
idx_26    fist              528     715        537      808       +9     +93       ✓
idx_27    fist              226     394        242      502      +16    +108       ✗
idx_28    fist              608     777        605      854       -3     +77       ✓
idx_29    fist              585     747        601      956      +16    +209       ✗
idx_30    fist              690     816        696      976       +6    +160       ✗
idx_31    fist              622     865        416      976     -206    +111       ✗
idx_32    fist              438     613        437      681       -1     +68       ✓
idx_33    fist              650     865        664      976      +14    +111       ✗
idx_34    fist              442     64

In [9]:
# ─── STEP 4: Cross-user validation on multiple users ──────────────────────────

print("=" * 80)
print("  CROSS-USER VALIDATION: Threshold detection vs Ground Truth")
print("=" * 80)

users_to_check = ['user1', 'user2', 'user3', 'user5', 'user10']
results_per_user = []

for user_name in users_to_check:
    try:
        user_data = load_user_json(TRAINING_PATH, user_name)
    except Exception:
        continue
    
    # Recompute rest noise floor per user (from their own noGesture samples)
    rest_vals = []
    for idx in range(1, 26):
        s = user_data['trainingSamples'][f'idx_{idx}']
        if s['gestureName'] == 'noGesture':
            rest_vals.append(compute_multichannel_rms_envelope(s))
    
    if not rest_vals:
        continue
    rest_all = np.concatenate(rest_vals)
    mu = np.mean(rest_all)
    sig = np.std(rest_all)
    th = mu + FACTOR_HIGH * sig
    tl = mu + FACTOR_LOW * sig
    
    user_matches = 0
    user_total = 0
    user_err_start = []
    user_err_end = []
    
    for idx in range(1, 151):
        s = user_data['trainingSamples'][f'idx_{idx}']
        gti = s.get('groundTruthIndex', [])
        if s['gestureName'] == 'noGesture' or len(gti) < 2:
            continue
        
        user_total += 1
        gt_s = gti[0] - 1
        gt_e = gti[1] - 1
        det_s, det_e = detect_gesture_double_threshold(s, th, tl)
        
        if det_s is not None:
            user_err_start.append(abs(det_s - gt_s))
            user_err_end.append(abs(det_e - gt_e))
            if abs(det_s - gt_s) < 100 and abs(det_e - gt_e) < 100:
                user_matches += 1
    
    match_pct = user_matches / user_total * 100 if user_total > 0 else 0
    mean_err_s = np.mean(user_err_start) if user_err_start else float('nan')
    mean_err_e = np.mean(user_err_end) if user_err_end else float('nan')
    
    results_per_user.append({
        'user': user_name, 'total': user_total, 'matches': user_matches,
        'pct': match_pct, 'mean_err_start': mean_err_s, 'mean_err_end': mean_err_e
    })
    
    print(f"\n{user_name}: {user_matches}/{user_total} matches ({match_pct:.1f}%)")
    print(f"  μ_rest={mu:.3f}, σ_rest={sig:.3f}, T_high={th:.3f}, T_low={tl:.3f}")
    print(f"  Mean |Δstart|={mean_err_s:.1f} samples ({mean_err_s/FS_EMG*1000:.0f} ms)")
    print(f"  Mean |Δend|  ={mean_err_e:.1f} samples ({mean_err_e/FS_EMG*1000:.0f} ms)")

# Overall summary
all_pcts = [r['pct'] for r in results_per_user]
print(f"\n{'='*80}")
print(f"OVERALL: Avg match rate across {len(results_per_user)} users: {np.mean(all_pcts):.1f}%")
print(f"  → Ground truth indices are {'ROUGHLY CONSISTENT' if np.mean(all_pcts) > 50 else 'INCONSISTENT'} "
      f"with energy-based detection.")
print(f"  Remaining discrepancy is expected due to the simplicity of the threshold "
      f"detector vs human annotation.")

  CROSS-USER VALIDATION: Threshold detection vs Ground Truth

user1: 37/125 matches (29.6%)
  μ_rest=2.765, σ_rest=0.584, T_high=5.101, T_low=3.933
  Mean |Δstart|=65.1 samples (326 ms)
  Mean |Δend|  =162.3 samples (812 ms)

user2: 25/125 matches (20.0%)
  μ_rest=2.679, σ_rest=0.997, T_high=6.667, T_low=4.673
  Mean |Δstart|=305.6 samples (1528 ms)
  Mean |Δend|  =176.7 samples (883 ms)

user3: 3/125 matches (2.4%)
  μ_rest=1.279, σ_rest=0.058, T_high=1.511, T_low=1.395
  Mean |Δstart|=421.8 samples (2109 ms)
  Mean |Δend|  =244.8 samples (1224 ms)

user5: 66/125 matches (52.8%)
  μ_rest=2.120, σ_rest=0.802, T_high=5.327, T_low=3.724
  Mean |Δstart|=59.8 samples (299 ms)
  Mean |Δend|  =154.0 samples (770 ms)

user10: 47/125 matches (37.6%)
  μ_rest=2.478, σ_rest=0.400, T_high=4.077, T_low=3.278
  Mean |Δstart|=246.8 samples (1234 ms)
  Mean |Δend|  =156.7 samples (784 ms)

OVERALL: Avg match rate across 5 users: 28.5%
  → Ground truth indices are INCONSISTENT with energy-based detect

In [10]:
# ─── STEP 5: Deeper diagnosis — is the START correct but the END overshoots? ──

print("=" * 90)
print("  DIAGNOSIS: Where does the mismatch come from?  (user1 detailed)")
print("=" * 90)

start_deltas = []
end_deltas = []
detected_count = 0

for idx in range(1, 151):
    s = training_user1['trainingSamples'][f'idx_{idx}']
    gti = s.get('groundTruthIndex', [])
    if s['gestureName'] == 'noGesture' or len(gti) < 2:
        continue
    gt_s = gti[0] - 1
    gt_e = gti[1] - 1
    det_s, det_e = detect_gesture_double_threshold(s, T_high, T_low)
    if det_s is not None:
        start_deltas.append(det_s - gt_s)
        end_deltas.append(det_e - gt_e)
        detected_count += 1

start_deltas = np.array(start_deltas)
end_deltas = np.array(end_deltas)

print(f"\nDetected gesture in {detected_count}/125 samples")

print(f"\n--- START BOUNDARY (detected - groundTruth) ---")
print(f"  Mean Δ:   {np.mean(start_deltas):+.1f} samples ({np.mean(start_deltas)/FS_EMG*1000:+.0f} ms)")
print(f"  Median Δ: {np.median(start_deltas):+.1f} samples ({np.median(start_deltas)/FS_EMG*1000:+.0f} ms)")
print(f"  Std:      {np.std(start_deltas):.1f} samples")
print(f"  Within ±50 samples (250ms): {np.sum(np.abs(start_deltas) < 50)}/{detected_count} "
      f"({np.sum(np.abs(start_deltas) < 50)/detected_count*100:.1f}%)")
print(f"  Within ±25 samples (125ms): {np.sum(np.abs(start_deltas) < 25)}/{detected_count} "
      f"({np.sum(np.abs(start_deltas) < 25)/detected_count*100:.1f}%)")

print(f"\n--- END BOUNDARY (detected - groundTruth) ---")
print(f"  Mean Δ:   {np.mean(end_deltas):+.1f} samples ({np.mean(end_deltas)/FS_EMG*1000:+.0f} ms)")
print(f"  Median Δ: {np.median(end_deltas):+.1f} samples ({np.median(end_deltas)/FS_EMG*1000:+.0f} ms)")
print(f"  Std:      {np.std(end_deltas):.1f} samples")
print(f"  Within ±50 samples (250ms): {np.sum(np.abs(end_deltas) < 50)}/{detected_count} "
      f"({np.sum(np.abs(end_deltas) < 50)/detected_count*100:.1f}%)")

print(f"\n--- INTERPRETATION ---")

start_ok = np.sum(np.abs(start_deltas) < 50) / detected_count * 100
end_positive = np.sum(end_deltas > 0) / detected_count * 100

print(f"  • START detection is {'GOOD' if start_ok > 60 else 'POOR'}: "
      f"{start_ok:.0f}% within ±250ms of ground truth")
print(f"  • END consistently overshoots: {end_positive:.0f}% of detected ends are AFTER the GT end")
print(f"  • Median overshoot at END: {np.median(end_deltas)/FS_EMG*1000:+.0f} ms")
print(f"\n  → The ground truth marks the 'active contraction' window.")
print(f"  → EMG energy persists AFTER the labeled gesture end (muscle relaxation phase).")
print(f"  → The ground truth indices appear to be CORRECTLY marking the intentional gesture,")
print(f"    NOT the full duration of elevated EMG activity.")
print(f"\n  CONCLUSION: Ground truth labels are reasonable. For feature extraction,")
print(f"  use groundTruthIndex directly — they mark the intended gesture window.")
print(f"  The 'extra' EMG after the label is relaxation, not part of the gesture.")

  DIAGNOSIS: Where does the mismatch come from?  (user1 detailed)

Detected gesture in 125/125 samples

--- START BOUNDARY (detected - groundTruth) ---
  Mean Δ:   -50.0 samples (-250 ms)
  Median Δ: +7.0 samples (+35 ms)
  Std:      149.5 samples
  Within ±50 samples (250ms): 103/125 (82.4%)
  Within ±25 samples (125ms): 101/125 (80.8%)

--- END BOUNDARY (detected - groundTruth) ---
  Mean Δ:   +127.9 samples (+640 ms)
  Median Δ: +126.0 samples (+630 ms)
  Std:      171.5 samples
  Within ±50 samples (250ms): 24/125 (19.2%)

--- INTERPRETATION ---
  • START detection is GOOD: 82% within ±250ms of ground truth
  • END consistently overshoots: 83% of detected ends are AFTER the GT end
  • Median overshoot at END: +630 ms

  → The ground truth marks the 'active contraction' window.
  → EMG energy persists AFTER the labeled gesture end (muscle relaxation phase).
  → The ground truth indices appear to be CORRECTLY marking the intentional gesture,
    NOT the full duration of elevated EMG 

## 13. Why do `startPointforGestureExecution` and `groundTruthIndex[0]` differ?

These two fields seem to measure different things. Let's investigate systematically.

In [11]:
import numpy as np

print("=" * 100)
print("  INVESTIGATING: startPointforGestureExecution vs groundTruthIndex")
print("=" * 100)

# ─── Hypothesis 1: Different sampling rates? ──────────────────────────────────
# EMG = 200 Hz, IMU (gyro/accel/quat) = 50 Hz.  Ratio = 4.
# If startPoint is in IMU samples, startPoint × 4 should ≈ groundTruthIndex[0]
print("\n--- Hypothesis 1: startPoint is in IMU (50 Hz) indices, GT is in EMG (200 Hz) indices ---")
print(f"{'Sample':<10} {'Gesture':<12} {'startPt':>8} {'startPt×4':>10} {'GTI[0]':>7} "
      f"{'Δ(×4-GT)':>9} {'GTI[1]':>7} {'EMG len':>8} {'IMU len':>8}")
print("-" * 100)

ratios = []
for idx_key in [f'idx_{i}' for i in range(26, 51)]:  # fist samples
    s = training_user1['trainingSamples'][idx_key]
    gti = s.get('groundTruthIndex', [])
    sp = s['startPointforGestureExecution']
    emg_len = len(s['emg']['ch1'])
    imu_len = len(s['gyroscope']['x'])
    
    if len(gti) >= 2:
        sp_scaled = sp * 4
        delta = sp_scaled - (gti[0] - 1)
        ratio = (gti[0] - 1) / sp if sp > 0 else float('nan')
        ratios.append(ratio)
        print(f"{idx_key:<10} {s['gestureName']:<12} {sp:>8} {sp_scaled:>10} {gti[0]:>7} "
              f"{delta:>+9} {gti[1]:>7} {emg_len:>8} {imu_len:>8}")

print(f"\nRatio GT[0]/startPt — mean: {np.mean(ratios):.3f}, "
      f"median: {np.median(ratios):.3f}, std: {np.std(ratios):.3f}")

# ─── Hypothesis 2: startPoint = CUE time, GT = actual muscle activation ──────
print("\n\n--- Hypothesis 2: startPoint = visual cue shown to user, GT = when muscle actually activates ---")
print("If this is the case, GT[0] should consistently come AFTER startPoint (reaction time).")

cue_vs_gt = []
for idx in range(26, 151):
    s = training_user1['trainingSamples'][f'idx_{idx}']
    gti = s.get('groundTruthIndex', [])
    if s['gestureName'] == 'noGesture' or len(gti) < 2:
        continue
    sp = s['startPointforGestureExecution']
    gt_start = gti[0] - 1  # 0-based
    cue_vs_gt.append({
        'gesture': s['gestureName'],
        'startPt': sp,
        'gt_start_emg': gt_start,
        'delay_samples': gt_start - sp,
        'startPt_x4': sp * 4,
        'delay_if_imu': gt_start - sp * 4,
    })

delays = np.array([x['delay_samples'] for x in cue_vs_gt])
delays_x4 = np.array([x['delay_if_imu'] for x in cue_vs_gt])

print(f"\nDirect comparison (startPt in EMG time, same as GT):")
print(f"  GT[0] - startPt:  mean={np.mean(delays):+.1f}, "
      f"median={np.median(delays):+.1f}, std={np.std(delays):.1f}")
print(f"  Always positive (GT after cue)? {np.all(delays > 0)}")
print(f"  Positive: {np.sum(delays > 0)}/{len(delays)}")

print(f"\nIf startPt is at 50 Hz (×4 to match EMG):")
print(f"  GT[0] - startPt×4: mean={np.mean(delays_x4):+.1f}, "
      f"median={np.median(delays_x4):+.1f}, std={np.std(delays_x4):.1f}")
print(f"  Always positive (GT after cue)? {np.all(delays_x4 > 0)}")
print(f"  Positive: {np.sum(delays_x4 > 0)}/{len(delays_x4)}")

# ─── Hypothesis 3: Check which one lines up with IMU length ──────────────────
print("\n\n--- Check: What is startPoint indexed relative to? ---")
s = training_user1['trainingSamples']['idx_26']
emg_len = len(s['emg']['ch1'])
imu_len = len(s['gyroscope']['x'])
sp = s['startPointforGestureExecution']
print(f"  Sample idx_26:")
print(f"    EMG length:  {emg_len}  (200 Hz)")
print(f"    IMU length:  {imu_len}  (50 Hz)")
print(f"    startPoint:  {sp}")
print(f"    startPoint < IMU length? {sp < imu_len}  → {'could be IMU index' if sp < imu_len else 'not IMU'}")
print(f"    startPoint < EMG length? {sp < emg_len}  → {'could be EMG index' if sp < emg_len else 'not EMG'}")
print(f"    GT[0]:       {s['groundTruthIndex'][0]}  (1-based)")
print(f"    GT length:   {len(s.get('groundTruth',[]))}  (should match EMG={emg_len})")

# ─── Check if startPoint aligns with myoDetection (50 Hz) ────────────────────
print(f"\n    myoDetection length: {len(s.get('myoDetection',[]))}  (should match IMU={imu_len})")
print(f"    startPoint {sp} < myoDetection length {len(s.get('myoDetection',[]))}? "
      f"{'Yes → likely IMU/50Hz index' if sp < len(s.get('myoDetection',[])) else 'No'}")

  INVESTIGATING: startPointforGestureExecution vs groundTruthIndex

--- Hypothesis 1: startPoint is in IMU (50 Hz) indices, GT is in EMG (200 Hz) indices ---
Sample     Gesture       startPt  startPt×4  GTI[0]  Δ(×4-GT)  GTI[1]  EMG len  IMU len
----------------------------------------------------------------------------------------------------
idx_26     fist              236        944     529      +416     716      996      250
idx_27     fist              173        692     227      +466     395      996      249
idx_28     fist              199        796     609      +188     778      992      250
idx_29     fist              608       2432     586     +1847     748     1000      250
idx_30     fist              309       1236     691      +546     817      996      249
idx_31     fist              527       2108     623     +1486     866      996      250
idx_32     fist              243        972     439      +534     614      992      249
idx_33     fist              632     

In [12]:
# ─── Definitive test: Is startPoint on the IMU timeline (50 Hz)? ──────────────
# If so, startPoint × (200/50) = startPoint × 4 = equivalent EMG index of the CUE.
# The delay from cue to actual muscle activation should be a positive "reaction time" (~200-500ms)

print("=" * 90)
print("  DEFINITIVE: startPoint is a CUE index. But at which sample rate?")
print("=" * 90)

# Test: startPoint appears consistently within [0, imu_length) for all samples
all_sps = []
all_imu_lens = []
all_emg_lens = []
for idx in range(1, 151):
    s = training_user1['trainingSamples'][f'idx_{idx}']
    sp = s['startPointforGestureExecution']
    imu_len = len(s['gyroscope']['x'])
    emg_len = len(s['emg']['ch1'])
    all_sps.append(sp)
    all_imu_lens.append(imu_len)
    all_emg_lens.append(emg_len)

all_sps = np.array(all_sps)
all_imu_lens = np.array(all_imu_lens)
all_emg_lens = np.array(all_emg_lens)

print(f"\nstartPoint range: [{all_sps.min()}, {all_sps.max()}]")
print(f"IMU length range: [{all_imu_lens.min()}, {all_imu_lens.max()}]")
print(f"EMG length range: [{all_emg_lens.min()}, {all_emg_lens.max()}]")
print(f"All startPoint < respective IMU len: {np.all(all_sps < all_imu_lens)}")
print(f"All startPoint < respective EMG len: {np.all(all_sps < all_emg_lens)}")
print(f"Any startPoint > max IMU len (250):  {np.any(all_sps > 250)}")

# Since startPoint values go up to ~630+ and IMU is ~250, it's NOT an IMU index!
# It must be an EMG index (200 Hz)
exceeds_imu = np.sum(all_sps >= all_imu_lens)
print(f"\nstartPoints EXCEEDING IMU length: {exceeds_imu}/{len(all_sps)}")

if exceeds_imu > 0:
    print(f"  → startPoint CANNOT be a 50 Hz index! Many values ({exceeds_imu}) exceed IMU length.")
    print(f"  → startPoint IS an EMG-rate (200 Hz) index.")
else:
    print(f"  → startPoint could be either IMU or EMG rate.")

# ─── So both are at 200 Hz. What's the delay? ────────────────────────────────
print(f"\n{'='*90}")
print(f"  Both startPoint and groundTruthIndex are at 200 Hz (EMG rate)")
print(f"{'='*90}")

# startPoint = when the CUE was shown on screen
# groundTruthIndex[0] = when the user's muscles actually started the gesture
# The difference = REACTION TIME

print(f"\n{'Gesture':<12} {'Count':>6} {'mean Δ':>9} {'median Δ':>10} {'std Δ':>8} "
      f"{'mean ms':>9} {'% GT>cue':>9}")
print("-" * 70)

gesture_delays = {}
for idx in range(26, 151):
    s = training_user1['trainingSamples'][f'idx_{idx}']
    gti = s.get('groundTruthIndex', [])
    if s['gestureName'] == 'noGesture' or len(gti) < 2:
        continue
    sp = s['startPointforGestureExecution']
    gt_s = gti[0] - 1  # 0-based
    g = s['gestureName']
    if g not in gesture_delays:
        gesture_delays[g] = []
    gesture_delays[g].append(gt_s - sp)

for g in sorted(gesture_delays.keys()):
    d = np.array(gesture_delays[g])
    pct_pos = np.sum(d > 0) / len(d) * 100
    print(f"{g:<12} {len(d):>6} {np.mean(d):>+9.1f} {np.median(d):>+10.1f} {np.std(d):>8.1f} "
          f"{np.mean(d)/200*1000:>+9.0f} {pct_pos:>8.0f}%")

all_delays = np.concatenate(list(gesture_delays.values()))
pct_pos_all = np.sum(all_delays > 0) / len(all_delays) * 100
print("-" * 70)
print(f"{'ALL':<12} {len(all_delays):>6} {np.mean(all_delays):>+9.1f} "
      f"{np.median(all_delays):>+10.1f} {np.std(all_delays):>8.1f} "
      f"{np.mean(all_delays)/200*1000:>+9.0f} {pct_pos_all:>8.0f}%")

print(f"\n--- INTERPRETATION ---")
print(f"  • startPointforGestureExecution: the EMG-rate (200 Hz) index when the VISUAL CUE")
print(f"    was shown to the user, telling them to perform the gesture.")
print(f"  • groundTruthIndex[0]: the EMG-rate index when the muscle ACTUALLY ACTIVATED,")
print(f"    as annotated (possibly by the Myo or manually).")
print(f"  • The mean delay is {np.mean(all_delays)/200*1000:+.0f} ms. Human reaction time is")
print(f"    typically 150-400 ms, but it can be negative if the annotation includes")
print(f"    preparatory muscle activity before the main contraction.")
print(f"  • {pct_pos_all:.0f}% of samples have GT start AFTER the cue (expected if GT = reaction).")
print(f"  • The large variance ({np.std(all_delays)/200*1000:.0f} ms) reflects natural variability")
print(f"    in human reaction time + annotation inconsistency.")

  DEFINITIVE: startPoint is a CUE index. But at which sample rate?

startPoint range: [166, 635]
IMU length range: [248, 252]
EMG length range: [990, 1008]
All startPoint < respective IMU len: False
All startPoint < respective EMG len: True
Any startPoint > max IMU len (250):  True

startPoints EXCEEDING IMU length: 118/150
  → startPoint CANNOT be a 50 Hz index! Many values (118) exceed IMU length.
  → startPoint IS an EMG-rate (200 Hz) index.

  Both startPoint and groundTruthIndex are at 200 Hz (EMG rate)

Gesture       Count    mean Δ   median Δ    std Δ   mean ms  % GT>cue
----------------------------------------------------------------------
fist             25    +105.7     +133.0    205.3      +529       76%
open             25      +0.8      +13.0    186.2        +4       60%
pinch            25     +41.0      +46.0    223.9      +205       68%
waveIn           25     +33.2      +59.0    211.2      +166       68%
waveOut          25     +46.6      +59.0    227.1      +233     

## 14. Are groundTruth labels from the Myo armband?

Comparing `myoDetection` (Myo's built-in classifier, 50 Hz) vs `groundTruth` (200 Hz) to determine their origin.

In [13]:
import numpy as np

print("=" * 95)
print("  COMPARING myoDetection vs groundTruth — are they the same thing?")
print("=" * 95)

# Check a few gesture samples in detail
for idx_key in ['idx_26', 'idx_51', 'idx_76', 'idx_101', 'idx_126']:
    s = training_user1['trainingSamples'][idx_key]
    gesture = s['gestureName']
    gti = s.get('groundTruthIndex', [])
    gt = s.get('groundTruth', [])
    myo = s.get('myoDetection', [])
    emg_len = len(s['emg']['ch1'])
    imu_len = len(s['gyroscope']['x'])
    
    print(f"\n--- {idx_key} | {gesture} ---")
    print(f"  EMG length:          {emg_len} (200 Hz)")
    print(f"  IMU length:          {imu_len} (50 Hz)")
    print(f"  groundTruth length:  {len(gt)} (same as EMG → 200 Hz)")
    print(f"  myoDetection length: {len(myo)} (same as IMU → 50 Hz)")
    print(f"  groundTruthIndex:    {gti} (1-based)")
    
    # Myo detection: what values and when?
    myo_arr = np.array(myo)
    unique_myo = np.unique(myo_arr)
    print(f"  myoDetection unique values: {unique_myo}")
    
    # Find where myoDetection is non-zero (i.e., Myo thinks a gesture is happening)
    myo_active = np.where(myo_arr != 0)[0]
    if len(myo_active) > 0:
        myo_start_imu = myo_active[0]
        myo_end_imu = myo_active[-1]
        # Convert to EMG time (×4)
        myo_start_emg = myo_start_imu * 4
        myo_end_emg = myo_end_imu * 4
        print(f"  myoDetection active: IMU idx [{myo_start_imu}, {myo_end_imu}] "
              f"→ EMG idx [{myo_start_emg}, {myo_end_emg}]")
        if len(gti) >= 2:
            gt_start = gti[0] - 1
            gt_end = gti[1] - 1
            print(f"  groundTruthIndex:    EMG idx [{gt_start}, {gt_end}]")
            print(f"  Δ start (myo - GT):  {myo_start_emg - gt_start:+d} EMG samples "
                  f"({(myo_start_emg - gt_start)/200*1000:+.0f} ms)")
            print(f"  Δ end   (myo - GT):  {myo_end_emg - gt_end:+d} EMG samples "
                  f"({(myo_end_emg - gt_end)/200*1000:+.0f} ms)")
    else:
        print(f"  myoDetection: all zeros (Myo didn't detect any gesture)")

# ─── Systematic comparison across all gesture samples ─────────────────────────
print(f"\n{'='*95}")
print(f"  SYSTEMATIC: myoDetection vs groundTruth across all 125 gesture samples")
print(f"{'='*95}")

myo_detected = 0
myo_missed = 0
start_deltas_myo = []
end_deltas_myo = []

for idx in range(26, 151):
    s = training_user1['trainingSamples'][f'idx_{idx}']
    gti = s.get('groundTruthIndex', [])
    myo = np.array(s.get('myoDetection', []))
    
    if len(gti) < 2:
        continue
    
    gt_start = gti[0] - 1
    gt_end = gti[1] - 1
    
    myo_active = np.where(myo != 0)[0]
    if len(myo_active) > 0:
        myo_detected += 1
        myo_s = myo_active[0] * 4
        myo_e = myo_active[-1] * 4
        start_deltas_myo.append(myo_s - gt_start)
        end_deltas_myo.append(myo_e - gt_end)
    else:
        myo_missed += 1

start_deltas_myo = np.array(start_deltas_myo)
end_deltas_myo = np.array(end_deltas_myo)

print(f"\n  Myo detected gesture: {myo_detected}/125")
print(f"  Myo MISSED gesture:   {myo_missed}/125")

if len(start_deltas_myo) > 0:
    print(f"\n  Start boundary (myo×4 − GT):")
    print(f"    Mean: {np.mean(start_deltas_myo):+.1f}, "
          f"Median: {np.median(start_deltas_myo):+.1f}, "
          f"Std: {np.std(start_deltas_myo):.1f} EMG samples")
    print(f"  End boundary (myo×4 − GT):")
    print(f"    Mean: {np.mean(end_deltas_myo):+.1f}, "
          f"Median: {np.median(end_deltas_myo):+.1f}, "
          f"Std: {np.std(end_deltas_myo):.1f} EMG samples")

print(f"\n{'='*95}")
print(f"  CONCLUSION")
print(f"{'='*95}")
print(f"""
  • groundTruth is at 200 Hz (same as EMG), myoDetection is at 50 Hz (same as IMU).
    → They are DIFFERENT signals from DIFFERENT sources.

  • myoDetection = Myo armband's BUILT-IN gesture classifier output.
    It classifies gestures in real-time at 50 Hz (values 0-5).

  • groundTruth / groundTruthIndex = DATASET AUTHORS' annotation.
    Looking at the official example code (preProcessing.py), the dataset authors
    use a SPECTROGRAM-BASED segmentation method (detectMuscleActivity):
      1. Compute spectrogram of summed EMG channels
      2. Sum spectrogram along frequency axis
      3. Threshold at 0.86
      4. Find onset/offset of activity
      5. Add 25-sample padding

  • The groundTruthIndex was likely generated using this or a similar algorithm
    during dataset creation — NOT from the Myo's built-in classifier.

  • The Myo's classifier (myoDetection) sometimes MISSES gestures entirely
    ({myo_missed}/125 missed), confirming they are independent.
""")

  COMPARING myoDetection vs groundTruth — are they the same thing?

--- idx_26 | fist ---
  EMG length:          996 (200 Hz)
  IMU length:          250 (50 Hz)
  groundTruth length:  996 (same as EMG → 200 Hz)
  myoDetection length: 250 (same as IMU → 50 Hz)
  groundTruthIndex:    [529, 716] (1-based)
  myoDetection unique values: [0 1]
  myoDetection active: IMU idx [157, 167] → EMG idx [628, 668]
  groundTruthIndex:    EMG idx [528, 715]
  Δ start (myo - GT):  +100 EMG samples (+500 ms)
  Δ end   (myo - GT):  -47 EMG samples (-235 ms)

--- idx_51 | open ---
  EMG length:          990 (200 Hz)
  IMU length:          249 (50 Hz)
  groundTruth length:  990 (same as EMG → 200 Hz)
  myoDetection length: 249 (same as IMU → 50 Hz)
  groundTruthIndex:    [520, 712] (1-based)
  myoDetection unique values: [0 4]
  myoDetection active: IMU idx [162, 175] → EMG idx [648, 700]
  groundTruthIndex:    EMG idx [519, 711]
  Δ start (myo - GT):  +129 EMG samples (+645 ms)
  Δ end   (myo - GT):  -11 E